In [2]:
import cohere
import os
import uuid
from typing import List, Dict

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(COHERE_API_KEY)

### Define and process documents

In [4]:
import requests

# Define the sources for the documents
# As an example, we'll use LLM University's Module 1: What are Large Language Models?
# https://docs.cohere.com/docs/intro-large-language-models

sources = [
    {
        "title": "Text Embeddings", 
        "url": "https://docs.cohere.com/docs/text-embeddings"},
    {
        "title": "Similarity Between Words and Sentences", 
        "url": "https://docs.cohere.com/docs/similarity-between-words-and-sentences"},
    {
        "title": "The Attention Mechanism", 
        "url": "https://docs.cohere.com/docs/the-attention-mechanism"},
    {
        "title": "Transformer Models", 
        "url": "https://docs.cohere.com/docs/transformer-models"}   
]

DEMO_CONNECTOR_API_KEY = "YOUR_CONNECTOR_API_KEY" # Replace with your connector API key

headers = {
    "Authorization": f"Bearer {DEMO_CONNECTOR_API_KEY}"
}

response = requests.post("YOUR_API_URL/process", # Replace with your API's URL
                         json={"sources": sources},
                         headers=headers)

print(response.json()["message"])

Documents processed successfully


In [9]:
import requests

# Define the sources for the documents
# As an example, we'll use LLM University's Module 1: What are Large Language Models?
# https://docs.cohere.com/docs/intro-large-language-models

sources = [
    {
        "title": "Text Embeddings", 
        "url": "https://docs.cohere.com/docs/text-embeddings"},
    {
        "title": "Similarity Between Words and Sentences", 
        "url": "https://docs.cohere.com/docs/similarity-between-words-and-sentences"},
    {
        "title": "The Attention Mechanism", 
        "url": "https://docs.cohere.com/docs/the-attention-mechanism"},
    {
        "title": "Transformer Models", 
        "url": "https://docs.cohere.com/docs/transformer-models"}   
]

DEMO_CONNECTOR_API_KEY = "6qXN7O-FdR9Q-89nW5UpbvRAk9q8oTgtc0EJz98yAg4" # Replace with your connector API key

headers = {
    "Authorization": f"Bearer {DEMO_CONNECTOR_API_KEY}"
}

response = requests.post("https://demo-conn.onrender.com/process", # Replace with your API's URL
                         json={"sources": sources},
                         headers=headers)

print(response.json()["message"])

Documents processed successfully


### Chatbot component

In [10]:
class Chatbot:
    def __init__(self, connectors: List[str]):
        self.conversation_id = str(uuid.uuid4())
        self.connectors = [{"id": c} for c in connectors]


    def generate_response(self, message: str):
        response = co.chat(
                        message=message,
                        connectors=self.connectors,
                        conversation_id=self.conversation_id,
                        stream=True
                        )

        for event in response:
                yield event

### App component

In [11]:
class App:
    def __init__(self, chatbot: Chatbot):
        self.chatbot = chatbot
        
    def run(self):
        while True:
            # Get the user message
            message = input("User: ")

            # Typing "quit" ends the conversation
            if message.lower() == "quit":
                print("Ending chat.")
                break
            else:
                print(f"User: {message}")

            # Get the chatbot response
            response = self.chatbot.generate_response(message)

            # Print the chatbot response
            print("Chatbot:")
            
            documents = []
            documents_flag = False
            citations_flag = False
            
            for event in response:
                # Documents
                if event.event_type == "search-results":
                    documents_flag = True
                    documents = event.documents
                    
                # Text
                if event.event_type == "text-generation":
                    print(event.text, end="")        

                # Citations
                if event.event_type == "citation-generation":
                    if not citations_flag:
                        print("\n\nCITATIONS:")
                        citations_flag = True
                    print(event.citations)
            
            if documents_flag:
                print("\n\nDOCUMENTS:")
                for d in documents:
                    print(f'{d["title"]} ({d["id"]}). URL: {d["url"]}')

            print(f"\n{'-'*100}\n")

### Run chatbot

In [13]:
# Define connectors
connectors = ["demo-conn-tm17qr"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What is attention
Chatbot:
Attention is a technique used in language models to provide context to each word in a sentence or text, based on the other words. Attention plays a crucial role in transformer models, which can help improve large language models.

CITATIONS:
[{'start': 60, 'end': 67, 'text': 'context', 'document_ids': ['demo-conn-tm17qr_0', 'demo-conn-tm17qr_1', 'demo-conn-tm17qr_2']}]
[{'start': 68, 'end': 102, 'text': 'to each word in a sentence or text', 'document_ids': ['demo-conn-tm17qr_1', 'demo-conn-tm17qr_2']}]
[{'start': 117, 'end': 129, 'text': 'other words.', 'document_ids': ['demo-conn-tm17qr_1']}]
[{'start': 148, 'end': 160, 'text': 'crucial role', 'document_ids': ['demo-conn-tm17qr_2']}]
[{'start': 164, 'end': 182, 'text': 'transformer models', 'document_ids': ['demo-conn-tm17qr_2']}]
[{'start': 199, 'end': 229, 'text': 'improve large language models.', 'document_ids': ['demo-conn-tm17qr_2']}]


DOCUMENTS:
Transformer Models (demo-conn-tm17qr_0). URL: http